# LA Q3 basic characterization

includes RR, ESR, Rabi, T1

## Header that we always run

In [6]:
# set up non-QM hardware
import Labber
# connect to server 
client = Labber.connectToServer('localhost') # get list of instruments 
#instruments = client.getListOfInstrumentsString() 
#for instr in instruments: 
#    print(instr) # close connection 

# reset all QDevil channels to 0 V
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))
for n in range(24):
    if n+1 < 10:
        QDevil.setValue("CH0" + str(n+1) + " Voltage", 0.0)
    else:
        QDevil.setValue("CH" + str(n+1) + " Voltage", 0.0)
# setting CH07 to sweet spot
DC_ss = 0
QDevil.setValue("CH07 Voltage", DC_ss)

# digital attenuators
Vaunix1 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25606'))
Vaunix2 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25607'))
ROI = 42
ROO = 10
Vaunix1.setValue("Attenuation", ROI)
Vaunix2.setValue("Attenuation", ROO)

# TWPA pump
SG = client.connectToInstrument('Rohde&Schwarz RF Source', dict(interface='TCPIP', address='192.168.88.2'))
freq_TWPA = 6730.6E6;
pwr_TWPA = -11.3;
SG.setValue('Frequency', freq_TWPA)
SG.setValue('Power', pwr_TWPA)

client.close()

In [25]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from configuration import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from qm import SimulationConfig
import qdac as qdac
from scipy.io import savemat
from scipy.optimize import curve_fit
%matplotlib qt
import datetime 
import os
import time

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)

## RR spectroscopy

In [3]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'RR_spec'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

qubit_index = 6
res_if = machine.resonators[qubit_index].f_readout - machine.resonators[qubit_index].lo
n_avg = 1000
cooldown_time =20_000 # convert to ns in the wait function!

# The QUA program #
RR_freq_sweep = np.arange(-5e6, +5e6, 0.1e6)

with program() as resonator_spec:
    n = declare(int)
    n_st = declare_stream()
    df = declare(int)
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        save(n, n_st)
        with for_(*from_array(df, RR_freq_sweep)):
            update_frequency(machine.resonators[qubit_index].name, df + res_if)
            measure(
                "readout"*amp(1),
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        n_st.save('iteration')
        I_st.buffer(len(RR_freq_sweep)).average().save("I")
        Q_st.buffer(len(RR_freq_sweep)).average().save("Q")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

# Simulate or execute #
simulate = False # simulation is useful to see the sequence, especially the timing (clock cycle vs ns)

if simulate:
    simulation_config = SimulationConfig(duration=1000)
    job = qmm.simulate(config, resonator_spec, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(resonator_spec)
    # Get results from QUA program
    res_handles = job.result_handles
    I_handles = res_handles.get("I")
    Q_handles = res_handles.get("Q")
    iteration_handles = res_handles.get("iteration")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [12, 8]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while job.result_handles.is_processing():
        # Fetch results
        iteration = res_handles.get("iteration").fetch_all()
        I = res_handles.get("I").fetch_all()
        Q = res_handles.get("Q").fetch_all()
        # progress bar
        progress_counter(iteration, n_avg)

2023-08-18 08:57:42,907 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [5]:
# Fetch results
iteration = res_handles.get("iteration").fetch_all()
I = res_handles.get("I").fetch_all()
Q = res_handles.get("Q").fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
idx = np.argmin(sigs) # find minimum
print(f"IF offset to add to IF: {RR_freq_sweep[idx] / u.MHz:.6f} MHz")

# 1D spectroscopy plot
fig = plt.figure(figsize=[8, 4])
plt.title("Resonator spectroscopy")
plt.plot((machine.resonators[qubit_index].f_readout + RR_freq_sweep ) / u.MHz, sigs, ".")
plt.xlabel("Frequency [MHz]")
plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
plt.axvline(x = (machine.resonators[qubit_index].f_readout + RR_freq_sweep[idx]) / u.MHz)

IF offset to add to IF: 0.300000 MHz


In [15]:
savemat(os.path.join(tPath, file_name), {"RR_freq": machine.resonators[qubit_index].f_readout + RR_freq_sweep, "sig": sigs, "DC_CH07": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [16]:
machine.resonators[qubit_index].f_readout += 0.0E6
machine._save("quam_state.json", flat_data=False)

## Flux sweep

In [33]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'RR_spec_flux_sweep'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

In [36]:
# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
# machine.resonators[0].f_readout+=1.68e6
# machine.resonators[0].readout_pulse_amp=0.27
config = build_config(machine)

# Program-specific variables #
n_avg =1000  # Number of averaging loops

cooldown_time = 20_000  # Resonator cooldown time, convert to ns in the wait() function 
flux_settle_time = 250  # Flux settle time for fast flux

flux_pts = 250
#dcs = np.linspace(-0.49, 0.49, flux_pts)
dc_flux_sweep = np.arange(-9,9 + 0.1,0.1)
RR_freq_sweep = np.arange(-12.5e6, 2.5e6, 0.05e6)

res_if = machine.resonators[qubit_index].f_readout - machine.resonators[qubit_index].lo

qubit_index = 6
client = Labber.connectToServer('localhost') # get list of instruments 
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))

start_time = time.time()
for idx_dc, _ in enumerate(dc_flux_sweep):
    dc_flux = dc_flux_sweep[idx_dc]
    QDevil.setValue("CH07 Voltage", dc_flux)
# The QUA program #
    time.sleep(0.2)

    with program() as resonator_spec_2D:
        n = declare(int)  # Averaging index
        df = declare(int)  # Resonator frequency
        # dc = declare(fixed)  # flux dc level|
        I = declare(fixed)
        Q = declare(fixed)
        I_st = declare_stream()
        Q_st = declare_stream()
        n_st = declare_stream()
        #wait(flux_settle_time , machine.resonators[qubit_index].name, machine.qubits[qubit_index].name)

        with for_(n, 0, n < n_avg, n + 1):
            with for_(*from_array(df, RR_freq_sweep)):
                # Update the resonator frequency
                update_frequency(machine.resonators[qubit_index].name, df + res_if)
                # with for_(*from_array(dc, dcs)):
                    # Flux sweeping
                    # set_dc_offset(machine.flux_lines[qubit_index].name, "single", dc)
                    # wait(flux_settle_time * u.ns, machine.resonators[qubit_index].name, machine.qubits[qubit_index].name)
                    # Measure the resonator
                measure(
                    "readout",
                    machine.resonators[qubit_index].name,
                    None,
                    dual_demod.full("cos", "out1", "sin", "out2", I),
                    dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                )
                # Wait for the resonator to cooldown
                wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
                # Save data to the stream processing
                save(I, I_st)
                save(Q, Q_st)
            save(n, n_st)

        with stream_processing():
            # I_st.buffer(len(dcs)).buffer(len(dfs)).average().save("I")
            # Q_st.buffer(len(dcs)).buffer(len(dfs)).average().save("Q")
            I_st.buffer(len(RR_freq_sweep)).average().save("I")
            Q_st.buffer(len(RR_freq_sweep)).average().save("Q")
            n_st.save("iteration")

    #  Open Communication with the QOP  #
    qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

    simulation = False

    qm = qmm.open_qm(config)
    job = qm.execute(resonator_spec_2D)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")

    fig = plt.figure(1)
    plt.rcParams['figure.figsize'] = [8, 4]
    interrupt_on_close(fig, job)  #  Interrupts the job when closing the figure

    # Progress bar
    progress_counter(idx_dc, len(dc_flux_sweep), start_time=start_time)

    while results.is_processing():
        I_tmp, Q_tmp, iteration_tmp = results.fetch_all()
        I_tmp = u.demod2volts(I_tmp, machine.resonators[qubit_index].readout_pulse_length)
        Q_tmp = u.demod2volts(Q_tmp, machine.resonators[qubit_index].readout_pulse_length)

        # Live plotting
        plt.cla()
        plt.title("Resonator spectroscopy")
        plt.plot((machine.resonators[qubit_index].f_readout + RR_freq_sweep ) / u.MHz, np.sqrt(I_tmp**2 + Q_tmp**2), ".")
        plt.xlabel("Frequency [MHz]")
        plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
        plt.pause(0.1)
        interrupt_on_close(fig, job)  # Interrupts the job when closing the figure

    if idx_dc == 0:
        I_2D = I_tmp
        Q_2D = Q_tmp
    else:
        I_2D = np.hstack([I_2D, I_tmp])
        Q_2D = np.hstack([Q_2D, Q_tmp])

sigs = np.sqrt(I_2D**2 + Q_2D**2)
savemat(os.path.join(tPath, file_name), {"RR_freq": machine.resonators[qubit_index].f_readout + RR_freq_sweep, "sig": sigs, "dc_flux_sweep": dc_flux_sweep, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

2023-08-18 09:49:40,337 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

2023-08-18 09:50:34,823 - qm - INFO     - Performing health check
2023-08-18 09:50:34,837 - qm - INFO     - Health check passed
2023-08-18 09:50:35,744 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 09:50:35,892 - qm - INFO     - Executing program
2023-08-18 09:50:52,953 - qm - ERROR    - Encountered connection error from QOP-> elapsed time: 1692377436.13s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-

2023-08-18 09:51:47,089 - qm - INFO     - Performing health check
2023-08-18 09:51:47,101 - qm - INFO     - Health check passed
2023-08-18 09:51:47,774 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 09:51:47,948 - qm - INFO     - Executing program
2023-08-18 09:52:04,988 - qm - ERROR    - Encountered connection error from QOP-> elapsed time: 1692377508.15s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-

2023-08-18 09:52:59,742 - qm - INFO     - Performing health check
2023-08-18 09:52:59,755 - qm - INFO     - Health check passed
2023-08-18 09:53:00,374 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 09:53:00,564 - qm - INFO     - Executing program
2023-08-18 09:53:17,545 - qm - ERROR    - Encountered connection error from QOP--> elapsed time: 1692377580.76s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber

2023-08-18 09:54:12,175 - qm - INFO     - Performing health check
2023-08-18 09:54:12,184 - qm - INFO     - Health check passed
2023-08-18 09:54:12,842 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 09:54:12,993 - qm - INFO     - Executing program
2023-08-18 09:54:29,999 - qm - ERROR    - Encountered connection error from QOP--> elapsed time: 1692377653.25s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber

...

2023-08-18 10:38:57,081 - qm - INFO     - Performing health check
2023-08-18 10:38:57,095 - qm - INFO     - Health check passed
2023-08-18 10:38:57,753 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 10:38:57,921 - qm - INFO     - Executing program
2023-08-18 10:39:14,844 - qm - ERROR    - Encountered connection error from QOP) --> elapsed time: 1692380338.13s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labb

2023-08-18 10:40:09,698 - qm - INFO     - Performing health check
2023-08-18 10:40:09,708 - qm - INFO     - Health check passed
2023-08-18 10:40:10,569 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 10:40:10,771 - qm - INFO     - Executing program
2023-08-18 10:40:27,851 - qm - ERROR    - Encountered connection error from QOP) --> elapsed time: 1692380410.85s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labb

2023-08-18 10:41:22,153 - qm - INFO     - Performing health check
2023-08-18 10:41:22,165 - qm - INFO     - Health check passed
2023-08-18 10:41:23,079 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 10:41:23,216 - qm - INFO     - Executing program
2023-08-18 10:41:40,292 - qm - ERROR    - Encountered connection error from QOP) --> elapsed time: 1692380483.44s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labb

2023-08-18 10:42:34,546 - qm - INFO     - Performing health check
2023-08-18 10:42:34,560 - qm - INFO     - Health check passed
2023-08-18 10:42:35,405 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 10:42:35,607 - qm - INFO     - Executing program
2023-08-18 10:42:52,565 - qm - ERROR    - Encountered connection error from QOP) --> elapsed time: 1692380555.82s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labb

2023-08-18 10:43:47,077 - qm - INFO     - Performing health check
2023-08-18 10:43:47,090 - qm - INFO     - Health check passed
2023-08-18 10:43:47,708 - qm - INFO     - Sending program to QOP for compilation
2023-08-18 10:43:47,878 - qm - INFO     - Executing program
2023-08-18 10:44:04,932 - qm - ERROR    - Encountered connection error from QOP) --> elapsed time: 1692380628.06s
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labb

In [37]:
client.close()

In [42]:
I_2D = np.reshape(I_2D, (len(dc_flux_sweep), len(I_tmp)))
Q_2D = np.reshape(Q_2D, (len(dc_flux_sweep), len(Q_tmp)))
# 2D plot
fig = plt.figure()
plt.rcParams['figure.figsize'] = [8, 4]
# 2D spectroscopy plot
plt.cla()
plt.title("Resonator spectroscopy tuning curve")
plt.pcolor((machine.resonators[qubit_index].f_readout + RR_freq_sweep)/ u.MHz, dc_flux_sweep, np.sqrt(I_2D**2 + Q_2D**2), cmap="seismic")
plt.ylabel("DC flux[V]")
plt.xlabel("Frequency [MHz]")
plt.colorbar()

## ESR sweet spot

In [ ]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'RR_spec_flux_sweep'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

In [ ]:
-0.111342